In [1]:
import sys
import os
import pandas as pd
import warnings
import re
import datetime

!mkdir 'RezoJDM16K-OpenKE'
!git clone https://github.com/ContentSide/RezoJDM16K.git

Cloning into 'RezoJDM-SDS'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 157 (delta 68), reused 132 (delta 50), pack-reused 0
Receiving objects: 100% (157/157), 32.68 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [18]:
triplets_train = pd.read_csv("/content/RezoJDM16K/datasets/Triplets/triplets_train.csv", index_col=False)
triplets_dev = pd.read_csv("/content/RezoJDM16K/datasets/Triplets/triplets_dev.csv", index_col=False)
triplets_test = pd.read_csv("/content/RezoJDM16K/datasets/Triplets/triplets_test.csv", index_col=False)

relation_names = pd.read_csv("/content/RezoJDM16K/datasets/Triplets/relations_ids_names.csv", index_col=False)
nodes_all = pd.read_csv("/content/RezoJDM16K/datasets/Filtered_Nodes_Edges/nodes.csv", index_col=False)

In [14]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

nodes = nodes_in_train.intersection(nodes_in_dev)
nodes = nodes.intersection(nodes_in_test)
nodes = list(nodes)

def remove_delta_nodes(triplets, nodes):
    return triplets[triplets["n1"].isin(nodes) & triplets["n2"].isin(nodes)].reset_index()

triplets_train = remove_delta_nodes(triplets_train, nodes)
triplets_dev = remove_delta_nodes(triplets_dev, nodes)
triplets_test = remove_delta_nodes(triplets_test, nodes)

In [25]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

nodes = nodes_in_train.union(nodes_in_dev)
nodes = nodes.union(nodes_in_test)
nodes = list(nodes)
len(nodes)

15746

In [23]:
nodes_in_train = set(triplets_train["n1"]).union(set(triplets_train["n2"]))
nodes_in_dev = set(triplets_dev["n1"]).union(set(triplets_dev["n2"]))
nodes_in_test = set(triplets_test["n1"]).union(set(triplets_test["n2"]))

len(nodes_in_train), len(nodes_in_dev), len(nodes_in_test)

(15746, 15615, 15623)

In [26]:
nodes_name = nodes_all[nodes_all["eid"].isin(nodes)].reset_index()[["eid", "n"]]
len(nodes_name), nodes_all.shape[0]

(15746, 73106)

In [27]:
nodes_name = nodes_name.rename_axis('new_eid').reset_index()
jdm2oke_node_mapping = {}
for _ , row in nodes_name.iterrows():
    jdm2oke_node_mapping[row.eid] = row.new_eid
len(jdm2oke_node_mapping)

15746

In [28]:
len(triplets_train["t"].unique()), len(triplets_dev["t"].unique()), len(triplets_test["t"].unique())

(53, 53, 53)

In [30]:
triplets_train.shape[0], triplets_dev.shape[0], triplets_test.shape[0]

(665675, 83209, 83209)

In [33]:
relation_names = relation_names[relation_names["relation_id"].isin(triplets_train["t"].unique())].reset_index()[["relation","relation_id"]]
relation_names = relation_names.rename_axis('new_relation_id').reset_index()
jdm2oke_relation_mapping = {}
for _ , row in relation_names.iterrows():
    jdm2oke_relation_mapping[row.relation_id] = row.new_relation_id
len(jdm2oke_relation_mapping)

53

In [34]:
def create_OpenKE_txt(triplets_csv, file_name):

    line = triplets_csv.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in triplets_csv.iterrows():
        text += (str(jdm2oke_node_mapping[row["n1"]]) +" " +str(jdm2oke_node_mapping[row["n2"]]) + " "+str(jdm2oke_relation_mapping[row["t"]]) +"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()

In [35]:
create_OpenKE_txt(triplets_train,'./RezoJDM16K-OpenKE/train2id.txt')
create_OpenKE_txt(triplets_dev,'./RezoJDM16K-OpenKE/valid2id.txt')
create_OpenKE_txt(triplets_test,'./RezoJDM16K-OpenKE/test2id.txt')

In [36]:
def create_OpenKE_ent(nodes_name, file_name):
    
    line = nodes_name.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in nodes_name.iterrows():
        text += (row["n"] +"\t" +str(row["new_eid"])+"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()


def create_OpenKE_rel(nodes_name, file_name):
    
    line = nodes_name.shape[0]
    text = ""

    text += (str(line) + "\n")

    for _, row in nodes_name.iterrows():
        text += (row["relation"] +"\t" +str(row["new_relation_id"])+"\n")
    text = text[:-1]

    output_file = open(file_name,'w')
    output_file.write(text)
    output_file.close()

In [37]:
create_OpenKE_ent(nodes_name, "./RezoJDM16K-OpenKE/entity2id.txt")
create_OpenKE_rel(relation_names, "./RezoJDM16K-OpenKE/relation2id.txt")

In [38]:
nodes_name

,new_eid,eid,n
0,0,1,vraiment
1,1,6,provenir
2,2,8,traversée
3,3,12,discréditer
4,4,16,Maria Callas
...,...,...,...
15741,15741,463699,chorioangiome
15742,15742,465567,Pierre Boulle
15743,15743,465579,Pierre Pelot
15744,15744,1449597,soldat>146050


In [39]:
!zip -r 'RezoJDM16K-OpenKE.zip' 'RezoJDM16K-OpenKE'

  adding: RezoJDM-SDS-OpenKE/ (stored 0%)
  adding: RezoJDM-SDS-OpenKE/test2id.txt (deflated 59%)
  adding: RezoJDM-SDS-OpenKE/relation2id.txt (deflated 49%)
  adding: RezoJDM-SDS-OpenKE/train2id.txt (deflated 68%)
  adding: RezoJDM-SDS-OpenKE/entity2id.txt (deflated 52%)
  adding: RezoJDM-SDS-OpenKE/valid2id.txt (deflated 59%)
